In [5]:
#1 - Import libraries o import data
'''this plugins is necessary
conda install -c conda-forge sunkit-instruments
sunpy.instr.goes was moved to sunkit-instruments
'''

import json
import urllib
import requests
import datetime as dt

import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mpld3

from sunpy.time import TimeRange
import sunkit_instruments.goes_xrs

import datetime
from sqlalchemy import Time


#creates directory to save data, if it does not exist
if not os.path.exists('data'):
    os.makedirs('data')



In [18]:
#2 - Set the date range to search for data and capture GOES events related to explosions

def capture_goes(start_date, end_date):

    #captures the data saved in the directory, otherwise, generates a new file with the events
    if os.path.isfile('data/goes_data.npy'):
        goes_events = np.load('data/goes_data.npy', allow_pickle=True, encoding='latin1')      
    else:
        print("Download new data GOES")
        time_range = TimeRange(start_date, end_date)
        goes_events = sunkit_instruments.goes_xrs.get_goes_event_list(time_range,"A1") #M1, means from M1 classes of solar flares

        print('GOES data captured and saved in data/goes_data.npy',len(goes_events),'events.')
        np.save("data/goes_data.npy", goes_events)

        list_goes = []

        for ge in goes_events:
            item_goes = []
            item_goes.append(ge['event_date'])
            item_goes.append(ge['start_time'])
            item_goes.append(ge['peak_time'])
            item_goes.append(ge['end_time'])
            item_goes.append(ge['goes_class'])
            item_goes.append(ge['goes_location'])
            item_goes.append(ge['noaa_active_region'])
            list_goes.append(item_goes)


        df = pd.DataFrame(list_goes)
        df.to_csv("data/goes_list.csv")


        
    # NOAA and hARPNUMS active regions map
    if os.path.isfile('data/all_harps_with_noaa_ars.txt'):
        num_mapper = pd.read_csv('data/all_harps_with_noaa_ars.txt', sep=' ')
    else:
        num_mapper = pd.read_csv('http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt',sep=' ')

    

    return goes_events, num_mapper

In [7]:
#3 - general functions for data conversion

def from_tai_time(tstr):
    """
    Convert the time string to a dt object
    """
    return dt.datetime.strptime(tstr, "%Y.%m.%d_%H:%M:%S_TAI")

def to_tai_time(date):
    """
    Convert from a dt object to a time string object
    """
    
    return date.strftime("%Y.%m.%d_%H:%M:%S_TAI")

def round_up(time):
    """
    Rounds up the time to the next time divisible by 12 minutes (JSOC data is saved every 12 minutes)
    
    Args:
    time := td object
    """
    #remainder = time.minute % 12
    remainder =  (int( str(time)[14:16] )) % 12

    if remainder != 0:
        time = time + dt.timedelta(minutes=12-remainder)
    return time

def convert_noaa_to_harpnum(noaa_ar, num_mapper):
    """
    Converts from a NOAA Active Region to a HARPNUM
    Returns harpnum if present, else None if there are no matching harpnums
    
    Args:
    """
    idx = num_mapper[num_mapper['NOAA_ARS'].str.contains(str(int(noaa_ar)))]
    return None if idx.empty else idx.HARPNUM.values[0]

In [9]:
#4 - Generates URL for search query and captures data
# formats for json queries to fetch the feature data and to check if the active region is close to the limb

def url_sharp():
    url_data = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s[{0}][{1}][? (CODEVER7 !~ '1.1 ')" + \
        "and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=DATE,DATE_S,DATE_B,DATE__OBS,DATE-OBS,T_OBS,T_REC,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,MEANJZD," + \
        "TOTUSJZ,MEANALP,MEANJZH,TOTUSJH,ABSNJZH,SAVNCPP,MEANPOT,TOTPOT,MEANSHR,SHRGT45,R_VALUE,AREA_ACR"


    ''' 
    these attributes mentioned in the original work come from the cgem_lorentz base and not hmi.sharp
    cgem_lorentz => base
    ,TOTBSQ,TOTFZ,EPSZ,TOTFY,TOTFX,EPSY,EPSX
    '''

    url_limb = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s[{0}][{1}][? (abs(OBS_VR)< 3500)" + \
        " and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"


    url_data = url_data.replace(" ", "%20")
    url_limb = url_limb.replace(" ", "%20")

    return url_data, url_limb

In [24]:
#5 - searches the data in the database by harpnum and checks if there is data related to it
def is_clean(harpnum, peak_time, time_deltas, url_data, url_limb):
    """
    Returns True is time/harpnum has the proper data, False otherwise
    
    Args:
    harpnum := the harpnum of this event
    peak_time := the peak_time of the event
    time_deltas :=a list of timedeltas, where each delta is a required time before the peak time
        the first delta is the time of the observation time
    """


    #create times needed
    t_recs = [to_tai_time(peak_time - td) for td in time_deltas]
    time = ",".join(t_recs)
    
    
    response_data = urllib.request.urlopen(url_data.format(harpnum, time))

    status_data = response_data.getcode()
        
    # if there's no response at this time, quit
    if status_data!= 200:
        print('skip: failed to find JSOC data')
        return False, data

    # read the JSON output
    data = json.loads(response_data.read())
    

    # if there is no data at this time, quit
    if data['count'] == 0:
        print('skip: there is no data for HARPNUM',harpnum,'for peak time', peak_time)
        return False, data

    if 'MISSING' in str(data['keywords']):
        print('skip: active region has some missing keywords')
        return False, data
    
    if len(data['keywords'][0]['values']) != len(time_deltas):
        print('skip: missing time points for HARPNUM',harpnum,'for peak time', peak_time)
        return False, data
    
    # check to see if the active region is too close to the limb
    # we can compute the latitude of an active region in stonyhurst coordinates as follows:
    # latitude_stonyhurst = CRVAL1 - CRLN_OBS
    # for this we have to query the CEA series 
    response_limb = urllib.request.urlopen(url_limb.format(harpnum, time))
    status_limb = response_limb.getcode()

    # if there's no response at this time, quit
    if status_limb!= 200:
        print('skip: failed to find CEA JSOC data')
        return False, data

    # read the JSON output
    latitude_information = json.loads(response_limb.read())

    # if there is no data at this time, quit
    if latitude_information['count'] == 0:
        print('skip: there is no data (lat) for HARPNUM',harpnum,'for peak time', peak_time)
        return False, data

    CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
    CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
    
    if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
        print('skip: latitude is out of range for',harpnum,'for peak time', peak_time)
        return False,data

    return True,data

In [25]:
#6 - search for positive class

def get_positive_class(time_deltas, noaa_events, num_mapper, url_data, url_limb):
    """
    Returns a positive class of harpnums/observation times
    
    Args:
    time_deltas := a list of timedeltas, where each delta is a required time before the peak time
        the first delta is the time of the observation time
    noaa_events := a list of noaa events that contain positive flares
    """
    pos_class = []
    json_pos_class = []
    jsontest = []

    for event in noaa_events:
        harpnum = convert_noaa_to_harpnum(event['noaa_active_region'], num_mapper)
        goes_class = event['goes_class']
        
        if harpnum == None:
            print('there are no matching HARPNUMs for', str(int(event['noaa_active_region'])))
            continue
              
            
        is_clean_teste, is_clean_data = is_clean(harpnum, event['peak_time'], time_deltas, url_data, url_limb)

    
        if is_clean_teste:
            pos_event = (harpnum, to_tai_time(event['peak_time']), event['goes_class'])
            pos_class.append(pos_event)
 
            ###################generate JSON with keywords ################################################################
            
            line = []
            for qt in range(5):

                list1 = {'harpnum': harpnum}
                line.append(list1)
               
                list1 = {'class_flare' : goes_class}
                line.append(list1)

                for qkey in range (len(is_clean_data['keywords'])): #scrolls through the number of times of keywords
                    pro = is_clean_data['keywords'][qkey]['name']
                    dad = is_clean_data['keywords'][qkey]['values'][qt]

                    list1 = {pro: dad}
                    line.append(list1)
                
                jsontest.append(line)
                line = []
            

    ################################### generate CSV with keywords + harpnum ###############################
    columnsd = []
    line = []
    linef = []

    for i in jsontest:
        columnsd = []
        for j in i:
            for key, value in j.items():
                columnsd.append(key)
                line.append(value)
        linef.append(line)
        line = []
        

    df = pd.DataFrame(linef, columns = [columnsd])
    df.to_csv (r'data/data_sharp_positive.csv', index = False, header=True)

    return pos_class

In [28]:
#7 get harpunum for time

def get_harpnum_from_time(time, max_td, time_deltas, url_data, url_limb):
    
    url_harpnum = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s[][{}]&op=rs_list&key=HARPNUM"

    datad = []
    temp = 0

    time_str = to_tai_time(time - max_td) + "," + to_tai_time(time) 
    response = urllib.request.urlopen(url_harpnum.format(time_str))
    status = response.getcode()

    # if there's no response at this time, quit
    if status!= 200:
        print('skip: failed to find JSOC data')
        return None, temp, datad

    # read the JSON output
    datad = json.loads(response.read())

    # if there is no data at this time, quit
    if datad['count'] == 0:
        print('skip: there is no data at time', time)
        return None, temp, datad
    
    harpnums = datad['keywords'][0]['values']
    
    if len(harpnums) == 0:
        print('skip: there are no harpnums at time', time)
        return None, temp, datad
    
    for harpnum in set(harpnums):
        if harpnums.count(harpnum) == 2:
            is_clean_test, is_clean_data = is_clean(harpnum, time, time_deltas)
            if is_clean_test:
                datad = is_clean_data
                return harpnum, time, datad
    
    return None, temp, datad

In [13]:
#8 - search negative classes

def get_negative_class(start_date, end_date, time_deltas, pos_events, max_size, url_data, url_limb):
    """
    Returns a negative class of harpnums/observation times
    
    Args:
    start_date := the date string in the form YYYY-MM-DD
    end_date := the date string in the form YYYY-MM-DD
    time_deltas := a list of timedeltas, where each delta is a required time before the peak time
        the first delta is the time of the observation time
    noaa_events := a list of noaa events that contain positive flares
    max_size := the maximum size of the negative class
    """
    ## get negative time range
    start = dt.datetime.strptime(start_date, '%Y-%m-%d')
    end = dt.datetime.strptime(end_date, '%Y-%m-%d')
    size = (end-start).days * 24 * 5

    
    all_times = {start + dt.timedelta(minutes=i*12) for i in range(size)}
    
   
    #remove times within the positive time range tirre .datetime apois peak_time
    peak_times = [round_up(event['peak_time']) for event in pos_events]

    for time in peak_times:
        positive_time = time - dt.timedelta(2)
        for _ in range(360):
            all_times.discard(positive_time)
            positive_time += dt.timedelta(minutes=12)
    
   
    # mix times to avoid any bias and decrease repeats
    negative_times = np.random.permutation(list(all_times))
    neg_class = []
    i = 0
    max_td = max(time_deltas)

    
    ###  imbalanced max_size = len(negative_times)

    jsontest = [] 

    # find viable negative events
    while len(neg_class) < max_size and i < len(negative_times):
        
        info, temp, info_data = get_harpnum_from_time(negative_times[i],max_td, time_deltas, url_data, url_limb)
        i += 1
        # if there are no harpnums at that time, quit
        if info == None:
            continue
        #harpnum,time = info
        harpnum = info
        neg_event = (harpnum, to_tai_time(time), None)
        neg_class.append(neg_event)
        
        
        ###################generate JSON with keywords #######################################################################

        line = []
        
        for qt in range(5):
            list1 = {'harpnum': harpnum}
            line.append(list1)
            
            list1 = {'class_flare' : "null"}
            line.append(list1)
            
            for qkey in range (len(info_data['keywords'])): #scrolls through the number of times of keywords
                pro = info_data['keywords'][qkey]['name']
                dad = info_data['keywords'][qkey]['values'][qt]

                list1 = {pro: dad}
                line.append(list1)
            
            jsontest.append(line)
            print("--------------------------------------------------ITEM JSON--------------------------------------------------")
            print(line)
            line = []

        
        ################################### generate CSV with keywords + o harpnum###############################
        columns = []
        line = []
        linef = []
        
        
        for i_json in jsontest:
            columns = []
            for j_json in i_json:
                for key, value in j_json.items():
                    columns.append(key)
                    line.append(value)
            linef.append(line)
            line = []
            
        

        df = pd.DataFrame(linef, columns = [columns])
        df.to_csv (r'data/data_sharp_negative.csv', index = False, header=True)
        ###################################################################################################################
        
    return neg_class

In [26]:
#9 - captures both positive and negative classes

def get_class_events(start_date, end_date, time_deltas, goes_events, num_mapper):
    """
    Returns a positive and negative class of harpnums/observation times
    
    Args:
    start_date := the date string in the form YYYY-MM-DD
    end_date := the date string in the form YYYY-MM-DD
    time_deltas := a list of timedeltas, where each delta is a required time before the peak time
        the first delta is the time of the observation time
    """
    # outputs, each element is a triple (harpnum, observation time, class)
    pos_class = []
    neg_class = []

    url_data, url_limb = url_sharp()
    
    ## gets the positive class
    print("Finding positive class ...")
    pos_class.extend(get_positive_class(time_deltas, goes_events, num_mapper, url_data, url_limb))
    
    
    ## get the negative class, for our purposes we will have a 1-to-1 ratio
    print("Finding negative class...")
    neg_class.extend(get_negative_class(start_date, end_date, time_deltas, goes_events, len(pos_class), url_data, url_limb))
    ###neg_class.extend(get_negative_class(start_date, end_date, time_deltas, goes_events, 200))
    
  
    
    return pos_class, neg_class

In [29]:
#10 - collects data, calling the other functions described above

# the times we need to have prior to the peak time
#required_times = [dt.timedelta(hours=i) for i in [6,12,18,24,30,36,42,48]] 
#required_times = [dt.timedelta(hours=i) for i in [0.2, 0.4, 0.6, 0.8, 1]] 
required_times = [dt.timedelta(hours=i) for i in [6,12,18,24]] 
start_date = '2010-05-01' #2010-05-01
end_date = '2010-06-01'   #2023-03-31

# collect data if it does not exist yet
if os.path.isfile('data/pos_class.npy') and os.path.isfile('data/neg_class.npy'):
    #add, allow_pickle=True, encoding='latin1'
    pos_class = np.load('data/pos_class.npy', allow_pickle=True, encoding='latin1')
    neg_class = np.load('data/neg_class.npy', allow_pickle=True, encoding='latin1')
else:

    goes_event, num_mapper = capture_goes(start_date,end_date)
    pos_class, neg_class = get_class_events(start_date, end_date, required_times, goes_event, num_mapper)
    np.save("data/pos_class", pos_class)
    np.save("data/neg_class", neg_class)



Finding positive class ...
skip: there is no data for HARPNUM 1 for peak time 2010-05-01 01:39:00.000
skip: there is no data for HARPNUM 2 for peak time 2010-05-01 05:27:00.000
skip: missing time points for HARPNUM 1 for peak time 2010-05-01 09:52:00.000
skip: missing time points for HARPNUM 1 for peak time 2010-05-02 06:09:00.000
skip: latitude is out of range for 1 for peak time 2010-05-02 10:09:00.000
skip: latitude is out of range for 1 for peak time 2010-05-02 15:14:00.000
skip: latitude is out of range for 1 for peak time 2010-05-02 16:26:00.000
skip: missing time points for HARPNUM 1 for peak time 2010-05-02 17:59:00.000


IndexError: list index out of range

In [ ]:
#REMOVE DUPLICATES FROM A CSV


import tensorflow as tf
import pandas as pd
import csv

file = tf.keras.utils
raw_df = pd.read_csv("data/RAW_DATA/2010-2023-02-28/Classes_A_B_C_M_X/consolidated-duplicate.csv", sep=";")
#raw_df.head()

raw_df = raw_df.drop_duplicates()

raw_df.head()

raw_df.to_csv("data/DADOS_BRUTOS/2010-2023-02-28/Classes_A_B_C_M_X/consolidated-duplicated-resolve.csv")


#import csv
#with open('dados/DADOS_BRUTOS/2010-2023-02-28/Classes_A_B_C_M_X/consolidado-duplicado.csv', 'w') as csvfile:
#    csv.writer(csvfile, delimiter=';').write(raw_df.head())



In [1]:
#balanceia dados selecionando a amostra negativa mais próxima da positiva
import tensorflow as tf
import pandas as pd
import csv

file = tf.keras.utils
raw_df = pd.read_csv("dados/DADOS_TREINAMENTO/2010-2023-03-abcmx.csv", sep=";")
linha_raw_df = raw_df[raw_df.columns[0]].count()
item0 = raw_df.iloc[0, :]

lista = []
lista_linhas_usadas = []

for i in range(linha_raw_df):
    
    linha_positiva = raw_df.iloc[i, :]
    print(linha_positiva)
    if linha_positiva.Class == 1:
        #adiciona linha positiva
        lista.append(linha_positiva)

        #avança proxima linha
        pode_avancar = i+i < linha_raw_df
        avanca_linha = True

        if(pode_avancar):
            linha_negativa = raw_df.iloc[i+1, :]  
            l_negativa = i+1
        else:
            if(i-1 >= 0):
                linha_negativa = raw_df.iloc[i-1,:]
                l_negativa = i - 1

        #checa se linha já foi usada
        while(avanca_linha):
            if l_negativa in lista_linhas_usadas or linha_negativa.harpnum != linha_positiva.harpnum or linha_negativa.Class == "1":
                if l_negativa + 1 < linha_raw_df:
                    linha_negativa = raw_df.iloc[l_negativa+1, :]
                    l_negativa = l_negativa + 1
                else:
                    #percorrer a lista de baixo para cima até o zero
                    linha_negativa = raw_df.iloc[l_negativa-1, :]
                    l_negativa = l_negativa - 1
            else:
                avanca_linha = False
                lista_linhas_usadas.append(l_negativa)
                lista.append(linha_negativa) #adiciona linha negativa
    

            


#cria dataframe novo com a lista e ger o csv
df_novo = pd.DataFrame(lista)
df_novo.to_csv("dados/DADOS_TREINAMENTO/2010-2023-03-abcmx_balanceado.csv")


c:\Users\Juliana\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3460: DtypeWarning: Columns (17,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


KeyboardInterrupt: 